In [2]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from umap import UMAP


import nltk
nltk.download('stopwords')

/Users/piyush/Desktop/dsml_Portfolio/new_project/venv/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/piyush/Desktop/dsml_Portfolio/new_project/venv/lib/python3.11/site-packages/umap/distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
/Users/piyush/Desktop/dsml_Portf

True

In [2]:
df = pd.read_csv('../data/paris_reviews.csv')


In [3]:
df.shape

(1406845, 6)

In [4]:
df['date'] = pd.to_datetime(df.date)
df['year'] = df['date'].dt.year



In [5]:
df.year.unique()

array([2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019,
       2020, 2021, 2022, 2023], dtype=int32)

In [6]:
df.year.value_counts()

year
2022    394577
2019    220245
2018    174277
2021    161016
2017    122830
2020     83678
2016     82636
2023     77870
2015     52798
2014     23088
2013      9167
2012      3386
2011      1031
2010       240
2009         6
Name: count, dtype: int64

In [7]:
df.shape

(1406845, 7)

In [8]:
df_orig = df.copy()

In [9]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)


def remove_urls(text):
    """Remove urls from a string"""
    import re
    return re.sub(r'http\S+', '', text, flags=re.MULTILINE)

def lowercase(text):
    """Remove urls from a string"""

    return ' '.join(x.lower() for x in text.split())


In [10]:
df['comments'] = df['comments'].astype(str)
df['comments'] = df['comments'].apply(remove_html_tags)
df['comments'] = df['comments'].apply(remove_urls)
df['comments'] = df['comments'].apply(lowercase)
df['date']= pd.to_datetime(df['date'])



#test = df.groupby('listing_id')['comments'].apply(list).reset_index(name='new')
#docs = test['new'].apply(lambda x : " ".join(x)).values
#df = df[df['year'] == 2023].reset_index(drop=True)

df = df[df['year'].isin([2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023])].reset_index(drop=True)
df = df.groupby('year').apply(lambda x: x.sample(5000,random_state=100)).reset_index(drop=True)

In [11]:
df.year.value_counts()

year
2013    5000
2014    5000
2015    5000
2016    5000
2017    5000
2018    5000
2019    5000
2020    5000
2021    5000
2022    5000
2023    5000
Name: count, dtype: int64

In [12]:
df

,listing_id,id,date,reviewer_id,reviewer_name,comments,year
0,599419,4592866,2013-05-14,5141358,Oksana,i booked lilian's apartment for my parents - t...,2013
1,1679711,8743133,2013-11-15,3674456,Wei Chian,it was a last minute reservation (day before) ...,2013
2,1780833,9186405,2013-12-11,1820498,Jeff,"what a wonderful flat. well-located, perfectly...",2013
3,111270,5275567,2013-06-22,4801752,Juliane,we had a wonderful time at francois' place. th...,2013
4,874864,7792037,2013-10-03,8083127,Danny,audrey is a wonderful host and her apartment i...,2013
...,...,...,...,...,...,...,...
54995,607213527992148908,810015717863921137,2023-01-22,305984574,Myriam,merci pour votre accueil.,2023
54996,24538105,828061132627903355,2023-02-16,220718628,Renata,"the stay was overall okay, get what we’ve paid...",2023
54997,54390678,815078458292521587,2023-01-29,326207448,Anna,a nice place to spend a few days in paris. jus...,2023
54998,25180580,795483123881965037,2023-01-02,191056209,Trin,guillaume is super wonderful host. we got a sh...,2023


In [13]:
#NLTK English stopwords
stopwords = nltk.corpus.stopwords.words('english')
# Print out the NLTK default stopwords
print(f'There are {len(stopwords)} default stopwords. They are {stopwords}')

There are 179 default stopwords. They are ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'no

In [14]:
names_1 = pd.read_csv('../data/app_c.csv',usecols=['name']).rename(columns={"name": "name"})
names_2 = pd.read_csv('../data/prenom.csv',usecols=['prenom']).rename(columns={"prenom": "name"})
names_3 = pd.read_csv('../data/Prenoms.csv',sep=';' , encoding='latin-1',usecols=['01_prenom']).rename(columns={"01_prenom": "name"})
names_4 = pd.read_csv('../data/patronymes.csv', usecols=['patronyme']).rename(columns={"patronyme": "name"})

names = pd.concat([names_1, names_2, names_3, names_4])
name_list = names['name'].str.lower().tolist()
names_list = list(map(( lambda x: str(x)+'s'), name_list))

In [15]:
airbnb_related_words = ['stay', 'airbnb', 'paris', 'would', 'time', 'apartment']

In [16]:
# Expand stopwords
stopwords.extend(name_list + names_list + airbnb_related_words)
print(f'There are {len(stopwords)} stopwords.')

There are 2504241 stopwords.


In [17]:
# Dimension reduction
from umap import UMAP
# Clustering
from hdbscan import HDBSCAN
# Count vectorization
from sklearn.feature_extraction.text import CountVectorizer
#from cuml.manifold import UMAP
#from cuml.cluster import HDBSCAN


# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Count vectorizer
vectorizer_model = CountVectorizer(stop_words=stopwords)

In [18]:
# Initiate UMAP
umap_model = UMAP(n_neighbors=15, 
                  n_components=5, 
                  min_dist=0.0, 
                  metric='cosine', 
                  random_state=100)
# Count vectorizer
vectorizer_model = CountVectorizer(stop_words=stopwords)

In [19]:
df.shape

(55000, 7)

In [20]:
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
representation_model = MaximalMarginalRelevance(diversity=0.8)

# Initiate BERTopic
topic_model = BERTopic(umap_model=umap_model, 
                       vectorizer_model=vectorizer_model, 
#                      min_topic_size=200,
#                       top_n_words=4,
                       language="multilingual",
                       calculate_probabilities=False,
                       representation_model=representation_model)



In [21]:
docs = df.comments
len(docs)

55000

In [22]:
import time
start = time.time()


# Run BERTopic model
topics = topic_model.fit_transform(docs)

end = time.time()
print(end - start)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [23]:
topic_model.get_topic_info().iloc[0:20]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,30030,-1_appartement_comfortable_recommend_bed,"[appartement, comfortable, recommend, bed, two...",[we had an amazing time at julieta’s apartment...
1,0,1604,0_spacieux_lumineux_emplacement_goût,"[spacieux, lumineux, emplacement, goût, confor...","[très bel appartement, propre et bien situé !,..."
2,1,976,1_using_bnb_airbnbs_booking,"[using, bnb, airbnbs, booking, next, bed, lite...",[this was one of the best air bnb experiences ...
3,2,948,2_propre_confortable_hôte_agencé,"[propre, confortable, hôte, agencé, commerces,...","[super séjour, calme, studio très propre et fo..."
4,3,906,3_recommend_beautiful_comfortable_equipped,"[recommend, beautiful, comfortable, equipped, ...",[the flat is brilliantly located if you're loo...
5,4,851,4_dirty_odeur_bathroom_chauffage,"[dirty, odeur, bathroom, chauffage, logement, ...",[les + emplacement parfait arrivée bien organi...
6,5,589,5_hôtel_déjeuner_literie_équipe,"[hôtel, déjeuner, literie, équipe, confortable...","[super hôtel, très bon week end à l’hôtel rege..."
7,6,542,6_parís_amazing_museums_distance,"[parís, amazing, museums, distance, cosy, reco...",[such a lovely place! it’s very petit but you ...
8,7,532,7_commerces_facilement_confortable_métros,"[commerces, facilement, confortable, métros, s...",[très bel appartement dans un quartier agréabl...
9,8,492,8_shopping_courtyard_wonderful_museums,"[shopping, courtyard, wonderful, museums, bed,...",[we had a wonderful time in paris staying in l...


In [ ]:
df.year.nunique()

In [ ]:
import time
start = time.time()


# Topics over time
topics_over_time = topic_model.topics_over_time(df['comments'], 
                                                df['date'], 
                                                global_tuning=True, 
                                              evolution_tuning=True,
                                                nr_bins=11)
# Take a look at the data
topics_over_time.head()


end = time.time()
print(end - start)

In [ ]:
topics_over_time.head()

In [ ]:
topics_over_time.Topic

In [ ]:
# Visualize topic over time
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=8)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, topics=[1, 2])

In [ ]:
topic_model.get_topic_info()

In [ ]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save("../model/paris/model_dir", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)




In [ ]:
loaded_model = BERTopic.load("/Users/piyush/Desktop/dsml_Portfolio/new_project/model/model_dir/")


In [74]:
drt = pd.read_parquet('../data/paris_reviews_preprocessed.parquet')

In [29]:
names_1 = pd.read_csv('../data/app_c.csv',usecols=['name']).rename(columns={"name": "name"})
names_2 = pd.read_csv('../data/prenom.csv',usecols=['prenom']).rename(columns={"prenom": "name"})
names_3 = pd.read_csv('../data/Prenoms.csv',sep=';' , encoding='latin-1',usecols=['01_prenom']).rename(columns={"01_prenom": "name"})
names_4 = pd.read_csv('../data/patronymes.csv', usecols=['patronyme']).rename(columns={"patronyme": "name"})

names = pd.concat([names_1, names_2, names_3, names_4])
name_list = names['name'].str.lower().tolist()
names_list = list(map(( lambda x: str(x)+'s'), name_list))

In [ ]:
        #NLTK English stopwords
        stopwords = nltk.corpus.stopwords.words('english')
        airbnb_related_words = ['stay', 'airbnb', 'paris', 'would', 'time', 'apartment']
        names_and_surnames = pd.read_csv('data/names_and_surnames.csv')

        # Expand stopwords
        stopwords.extend(names_and_surnames + airbnb_related_words)

In [64]:
import pandas as pd
import numpy as np
import glob
import os
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic
from bertopic.representation import MaximalMarginalRelevance
from umap import UMAP
import nltk
nltk.download('stopwords')



def train_bertopic(data):
    try:
        # Initiate UMAP
        umap_model = UMAP(n_neighbors=15, 
                        n_components=5, 
                        min_dist=0.0, 
                        metric='cosine', 
                        random_state=100)
        


        #NLTK English stopwords
        stopwords = nltk.corpus.stopwords.words('english')
        airbnb_related_words = ['stay', 'airbnb', 'paris', 'would', 'time', 'apartment']
        names_and_surnames = pd.read_csv('../data/names_and_surnames.csv')
        # Expand stopwords
        stopwords.extend(list(names_and_surnames['names_&_surnames']) + airbnb_related_words)


        vectorizer_model = CountVectorizer(stop_words=stopwords)
        representation_model = MaximalMarginalRelevance(diversity=0.8)

        # Initiate BERTopic
        topic_model = BERTopic(umap_model=umap_model, 
                            vectorizer_model=vectorizer_model, 
        #                      min_topic_size=200,
        #                       top_n_words=4,
                            language="multilingual",
                            calculate_probabilities=True,
                            representation_model=representation_model)



        import time
        start = time.time()


        # Run BERTopic model
        topics,_ = topic_model.fit_transform(data)

        end = time.time()
        print(end - start)

        # Return the trained model and topics
        return topic_model, topics
    
    except Exception as e:
        raise Exception(f"Error occurred during BERTopic training: {e}")
    


def main():
    parquet_file = "data/paris_reviews_preprocessed.parquet"

    try:
        # Process the Parquet file
        df = pd.read_parquet(parquet_file)
        docs = df.comments

        # Train the BERTopic model
        model, topics = train_bertopic(docs)

        # Print the topics or perform further analysis
        print(topics)

    except Exception as e:
        print(f"Error occurred during data processing: {e}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/piyush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [65]:
    parquet_file = "../data/paris_reviews_preprocessed.parquet"

In [66]:
        df = pd.read_parquet(parquet_file)
        docs = df.comments

In [67]:
len(docs)

55000

In [68]:
docs

0        i booked lilian's apartment for my parents - t...
1        it was a last minute reservation (day before) ...
2        what a wonderful flat. well-located, perfectly...
3        we had a wonderful time at francois' place. th...
4        audrey is a wonderful host and her apartment i...
                               ...                        
54995                            merci pour votre accueil.
54996    the stay was overall okay, get what we’ve paid...
54997    a nice place to spend a few days in paris. jus...
54998    guillaume is super wonderful host. we got a sh...
54999                                                ótimo
Name: comments, Length: 55000, dtype: object

In [69]:
model, topics = train_bertopic(docs[0:100])

9.9947509765625


In [70]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,78,-1_bed_recommend_stayed_métro,"[bed, recommend, stayed, métro, cosy, food, co...",[the flat is brilliantly located if you're loo...
1,0,12,0_recommend_could_breakfast_romantic,"[recommend, could, breakfast, romantic, bed, s...",[i booked lilian's apartment for my parents - ...
2,1,10,1_recommend_distance_ppls_wifi,"[recommend, distance, ppls, wifi, courtyard, b...",[apartment is small but is as described. walls...


In [72]:
topics

[0,
 -1,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 1,
 -1,
 -1,
 -1,
 -1,
 0,
 1,
 0,
 0,
 -1,
 -1,
 -1,
 1,
 0,
 0,
 -1,
 0,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 -1,
 0,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]

In [73]:
len(topics)

100